In [1]:
import pandas as pd

In [2]:
# Load the already merged minute-level data
df = pd.read_csv("data/processed/XAUUSD/combined_data.csv", parse_dates=['timestamp'], index_col='timestamp')
if 'ATR' in df.columns:
    df.drop(columns='ATR', inplace=True)

# Resample to daily OHLC
daily_ohlc = df.resample('1D').agg({
    'high': 'max',
    'low': 'min',
    'close': 'last'
})

# Calculate previous day's close
daily_ohlc['prev_close'] = daily_ohlc['close'].shift(1)

# Calculate True Range (TR)
daily_ohlc['true_range'] = daily_ohlc.apply(
    lambda row: max(
        row['high'] - row['low'],
        abs(row['high'] - row['prev_close']),
        abs(row['low'] - row['prev_close'])
    ) if pd.notnull(row['prev_close']) else None,
    axis=1
)

#Calculate 14-day ATR (excluding current day via shift)
daily_ohlc['atr_14'] = daily_ohlc['true_range'].rolling(window=14).mean().shift(1)

#Calculate moving averages on daily close (also shifted)
daily_ohlc['ma_14'] = daily_ohlc['close'].rolling(window=14).mean().shift(1)
daily_ohlc['ma_30'] = daily_ohlc['close'].rolling(window=30).mean().shift(1)
daily_ohlc['ma_100'] = daily_ohlc['close'].rolling(window=100).mean().shift(1)

# Rolling max/min of the daily close
daily_ohlc['max_14'] = daily_ohlc['close'].rolling(window=14).max().shift(1)
daily_ohlc['min_14'] = daily_ohlc['close'].rolling(window=14).min().shift(1)

daily_ohlc['max_30'] = daily_ohlc['close'].rolling(window=30).max().shift(1)
daily_ohlc['min_30'] = daily_ohlc['close'].rolling(window=30).min().shift(1)

daily_ohlc['max_100'] = daily_ohlc['close'].rolling(window=100).max().shift(1)
daily_ohlc['min_100'] = daily_ohlc['close'].rolling(window=100).min().shift(1)

# Forward-fill daily values into minute-level data
df['daily_high'] = daily_ohlc['high'].reindex(df.index, method='ffill')
df['daily_low'] = daily_ohlc['low'].reindex(df.index, method='ffill')
df['daily_close'] = daily_ohlc['close'].reindex(df.index, method='ffill')
df['true_range'] = daily_ohlc['true_range'].reindex(df.index, method='ffill')
df['atr_14'] = daily_ohlc['atr_14'].reindex(df.index, method='ffill')
df['ma_14'] = daily_ohlc['ma_14'].reindex(df.index, method='ffill')
df['ma_30'] = daily_ohlc['ma_30'].reindex(df.index, method='ffill')
df['ma_100'] = daily_ohlc['ma_100'].reindex(df.index, method='ffill')

#Day and Weeknumber
df['day_of_week'] = df.index.day_name().str[:3]
df['week_number'] = df.index.isocalendar().week

df['max_price_14'] = daily_ohlc['max_14'].reindex(df.index, method='ffill')
df['min_price_14'] = daily_ohlc['min_14'].reindex(df.index, method='ffill')

df['max_price_30'] = daily_ohlc['max_30'].reindex(df.index, method='ffill')
df['min_price_30'] = daily_ohlc['min_30'].reindex(df.index, method='ffill')

df['max_price_100'] = daily_ohlc['max_100'].reindex(df.index, method='ffill')
df['min_price_100'] = daily_ohlc['min_100'].reindex(df.index, method='ffill')

# Overwrite the original file
df.to_csv("data/processed/XAUUSD/combined_data.csv")

True Range, 14-day ATR, and 14/30/100-day MAs added — combined_data.csv has been overwritten.
